In [1]:
!apt-get install poppler-utils
!pip install pdf2image
!pip install pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
from pdf2image import convert_from_path
from PIL import Image
from os import listdir
from os.path import splitext
from pathlib import Path

In [3]:
Path("extract").mkdir(parents=True, exist_ok=True)

In [4]:
target_directory = './input'
image_exts = ['.png', '.jpg']
pdf_exts = ['.pdf']
target_ext = '.png'

for file in listdir(target_directory):
  filename, ext = splitext(file)
  try:
    if ext in image_exts:
      im = Image.open(f"{target_directory}/{filename}{ext}")
      im.save(f'extract/{filename}{target_ext}')
    if ext in pdf_exts:
      pages = convert_from_path(f"{target_directory}/{filename}{ext}")
      for i, page in enumerate(pages):
        page.save(f'extract/{filename}_page_{i}{target_ext}')
  except OSError:
    print('Cannot convert %s' % file)

In [6]:
!pip install opencv-python

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from random import randint

In [8]:
from google.colab.patches import cv2_imshow

In [9]:
def detect_black_squares(image):
  """
  Detect all black squares in the image
  Detection is based on detecting all squares,
  then selecting the largest one (max area)
  and filtering those that are similar to it.
  """
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  edged = cv2.Canny(binary, 30, 200)

  contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  black_square_coords = []
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      ratio = float(w/h)
      if ratio >= 0.95 and ratio <= 1.05: # +/- 5%
        black_square_coords.append((w * h, contour))
  black_square_coords.sort(key=lambda x: x[0], reverse=True)
  reference_area = black_square_coords[0][0]

  filtered_squares = [square for square in black_square_coords if abs(square[0] - reference_area) <= 0.05 * reference_area] # +/- 5%
  return filtered_squares

In [10]:
def detect_rotation_angle(contour):
  """
  Ratation detection
  Checking by what angle the image should be rotated
  so that the square is parallel to the edge
  """
  rect = cv2.minAreaRect(contour)
  angle = rect[2]
  return angle if angle < 90.0 else angle - 90.0

def rotate_image(image, angle):
  """
  Rotate the image by a given angle
  """
  size_reverse = np.array(image.shape[1::-1]) # swap x with y
  M = cv2.getRotationMatrix2D(tuple(size_reverse / 2.), angle, 1.)
  MM = np.absolute(M[:,:2])
  size_new = MM @ size_reverse
  M[:,-1] += (size_new - size_reverse) / 2.
  return cv2.warpAffine(image, M, tuple(size_new.astype(int)), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [11]:
def check_horizontal_line(squares, bottom_left_square):
  """
  Check the number of squares in a horizontal line to a given square
  """
  count = 0
  if bottom_left_square:
    _, y, _, h = bottom_left_square
    m = y + (h / 2)
    for square in squares:
      s_m = square[1] + (square[3] / 2)
      if m - 2 * h <= s_m <= m + 2 * h:
        count += 1
  return count

def check_vertical_line(squares, bottom_left_square):
  """
  Check the number of squares in a vertical line to a given square
  """
  count = 0
  if bottom_left_square:
    x, _, w, _ = bottom_left_square
    m = x + (w / 2)
    for square in squares:
      s_m = square[0] + (square[2] / 2)
      if m - 2 * w <= s_m <= m + 2 * w:
        count += 1
  return count

def find_bottom_left_square(squares, image):
  """
  Find left bottom square
  """
  height, _, _ = img.shape
  nearest_square = None
  min_distance = float('inf')
  for square in squares:
      x, y, _, _ = square
      distance = ((0 - x) ** 2 + (height - y) ** 2) ** 0.5
      if distance < min_distance:
          min_distance = distance
          nearest_square = square
  return nearest_square

def find_extreme_squares(squares, image):
  """
  Find the square closest to the top left
  and bottom right corners of the image
  """
  nearest_square = None
  farest_square = None
  min_distance = float('inf')
  max_distance = float('-inf')
  for square in squares:
      x, y, _, _ = square
      distance = x + y # L1
      if distance < min_distance:
          min_distance = distance
          nearest_square = square
      if distance > max_distance:
          max_distance = distance
          farest_square = square
  return nearest_square, farest_square

In [12]:
from pathlib import Path
Path("rotate").mkdir(parents=True, exist_ok=True)

for file in listdir('./extract'):
  print("Image:", file)
  img = cv2.imread(f"extract/{file}") # load image

  contours = detect_black_squares(img) # detect squares

  # calculate angle for each square
  angles = []
  for _, contour in contours:
    _angle = detect_rotation_angle(contour)
    angles.append(_angle)

  angle = np.average(angles) # calculating the final offset as an average
  print("Angle:", angle, angles)
  img = rotate_image(img, angle) # rotate image
  contours = detect_black_squares(img) # detect squares again (after rotation)

  squares = []
  for _, contour in contours:
    squares.append(cv2.boundingRect(contour))

  bottom_left_square = find_bottom_left_square(squares, img)
  horizontal = check_horizontal_line(squares, bottom_left_square)
  vertical = check_vertical_line(squares, bottom_left_square)

  nearest, farest = find_extreme_squares(squares, img)
  # crop image
  img = img[nearest[1] - nearest[3]:farest[1] + 2 * farest[3], nearest[0] - nearest[2]:farest[0] + 2 * farest[2]]

  # detect position based on pattern and rotate image
  if horizontal == 2 and vertical == 2:
    img = cv2.rotate(img, cv2.ROTATE_180)
  if horizontal == 3 and vertical == 2:
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  if horizontal == 2 and vertical == 3:
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)



  cv2.imwrite(f"rotate/{file}", img)

Image: 0
Angle: 70.21597544352214 [70.30797576904297, 70.13084411621094, 70.17753601074219, 70.09625244140625, 70.20112609863281, 70.38211822509766]
Image: 1
Angle: 36.22591908772787 [36.30449295043945, 36.145694732666016, 36.218833923339844, 36.179622650146484, 36.076011657714844, 36.43085861206055]
Image: 2
Angle: 9.255888144175211 [9.366509437561035, 9.188837051391602, 9.22988510131836, 9.462322235107422, 9.229887008666992, 9.05788803100586]
Image: 3
Angle: 60.23323885599772 [60.31281661987305, 60.2551155090332, 60.21453857421875, 60.113468170166016, 60.3269157409668, 60.176578521728516]
Image: 4
Angle: 21.25418535868327 [21.326217651367188, 21.317913055419922, 21.214115142822266, 21.169137954711914, 21.10483741760254, 21.39289093017578]
Image: 5
Angle: 2.188169082005819 [2.21679425239563, 2.2647545337677, 2.070030689239502, 2.2906100749969482, 2.21679425239563, 2.070030689239502]
Image: 6
Angle: 84.18722152709961 [84.2894058227539, 84.05313110351562, 84.1175537109375, 84.1273727416

In [13]:
def detect_student_name(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  edged = cv2.Canny(binary, 30, 200)

  contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  original_ratio = float(870 / 100) # hardcoded ratio based on sheet
  squares = []
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      ratio = float(w / h)
      if 0.8 * ratio <= original_ratio <= 1.2 * ratio:
        squares.append((w * h, (x, y, w, h)))

  squares.sort(key=lambda x: x[0], reverse=True)

  # find the pair of largest rectangles that fit the ratio
  similar_rectangles = None
  largest_area_difference = float('inf')

  for i in range(len(squares)):
      for j in range(i + 1, len(squares)):
          if 0.9 * squares[i][0] <= squares[j][0] <= 1.1 * squares[i][0]:
              area_difference = abs(squares[i][0] - squares[j][0])
              if area_difference < largest_area_difference:
                  similar_rectangles = (squares[i][1], squares[j][1])
                  largest_area_difference = area_difference

  # take the square that is lower than the other one
  rectangle = similar_rectangles[0] if similar_rectangles[0][1] > similar_rectangles[1][1] else similar_rectangles[1]
  assert rectangle is not None

  # put the mask on the rectangle
  x, y, w, h = rectangle
  image = cv2.rectangle(
    image,
    (x, y),
    (x + w, y + h),
    (255, 255, 255),
    -1
  )

  return rectangle

In [14]:
from pathlib import Path
Path("anonymize").mkdir(parents=True, exist_ok=True)

for file in listdir('./rotate'):
  print("Image:", file)
  img = cv2.imread(f"rotate/{file}")

  detect_student_name(img)

  cv2.imwrite(f"anonymize/{file}", img)

Image: 0
Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10
Image: 11
Image: 12
Image: 13
Image: 14
Image: 15
Image: 16
Image: 17
Image: 18
Image: 19
Image: 20


In [15]:
!zip -r /content/anonymize.zip /content/anonymize


  adding: content/anonymize/ (stored 0%)
  adding: content/anonymize/14.png (deflated 14%)
  adding: content/anonymize/5.png (deflated 15%)
  adding: content/anonymize/3.png (deflated 14%)
  adding: content/anonymize/15.png (deflated 15%)
  adding: content/anonymize/2.png (deflated 14%)
  adding: content/anonymize/20.png (deflated 16%)
  adding: content/anonymize/18.png (deflated 16%)
  adding: content/anonymize/4.png (deflated 14%)
  adding: content/anonymize/12.png (deflated 14%)
  adding: content/anonymize/0.png (deflated 14%)
  adding: content/anonymize/10.png (deflated 15%)
  adding: content/anonymize/8.png (deflated 14%)
  adding: content/anonymize/13.png (deflated 14%)
  adding: content/anonymize/9.png (deflated 15%)
  adding: content/anonymize/1.png (deflated 14%)
  adding: content/anonymize/19.png (deflated 16%)
  adding: content/anonymize/17.png (deflated 16%)
  adding: content/anonymize/6.png (deflated 14%)
  adding: content/anonymize/11.png (deflated 14%)
  adding: content/